In [10]:
import numpy as np 
from copy import copy
from numpy.linalg import norm
from scipy.sparse import csc_matrix, csr_matrix
from scipy.sparse.linalg import spsolve, cg, splu, gmres

In [26]:
def iterRef(A, b, tol=1e-5, maxiter=100, verbose=True):
    """
    Solve the equation a x = b for x using Iterative Refinement method.

    :param A: [(M,M) array_like] A square matrix.
    :param b: [(M,) array like]Right-hand side matrix in a x = b.
    :param maxIter: [int] max number of iteration for convergence.
    :param t:  #!
    :return x: (M,) or (M, N) ndarray
        Solution to the system a x = b. Shape of the return matches the shape of b.

    **Reference:**
        Burden, R.L. and Faires, J.D., 2011. Numerical analysis.
    """

    
    # declarations
    n = len(b)
    xx = np.zeros_like(b)
    r = np.zeros_like(b)

    lu = splu(A)
    x = lu.solve(b)
    res = np.sum(A.dot(x)-b)
    print("res :: A * x - b = {:e}".format(res))

    # check if converged
    if np.abs(res) < tol:
        print("IR ::: A * x - b = {:e}".format(res))
        return x

    k = 1                                       # step 1
    while (k <= maxiter):                       # step 2
        r = b - A.dot(x)                        # step 3
        y = lu.solve(r)                         # step 4
        xx = copy(x + y)                        # step 5

        if k == 1:                              # step 6
            # t = 16
            # COND = np.linalg.norm(y)/np.linalg.norm(xx) * 10**t
            # print("cond is ::::", COND)
            COND = np.linalg.cond(A.toarray())

        norm_ = np.linalg.norm(x-xx) # * np.linalg.norm(x) * 1e10
        print("iteration {:3d}, norm = {:e}".format(k, norm_))

        if norm_ < tol:                         # step 7

            if verbose:
#                 print("Conditional number of matrix A is: {:e}".format(COND))
                print("The procedure was successful.")
                print("IR: A * x - b = {:e}".format(np.sum(A.dot(xx)-b)))
                print(f"number of iteration is : {k:d}")
                print(" ")
            return xx

        k += 1                                 # step 8
        x = copy(xx)                           # step 9

    print("Max iteration exceeded.")
    print("The procedure was not successful.")
    print("Conditional number of matrix A is: {:e}".format(COND))
    print(" ")

    return None

In [27]:
def show_results(x, A, b, method):

#     print("solution is = ")
#     for i in x:
#         print("{:25.16e}".format(i))
    print("for {:16s} Ax-b is {:e}".format(
        method,
        norm(A.dot(x)-b)))

# LOAD DATA
def load_data(A_name, B_name):
    A = np.loadtxt(A_name, dtype=np.float64)
    B = np.loadtxt(B_name, dtype=np.float64)
    
    return A, B

In [28]:
A, B = load_data("data/A_large.txt", "data/B_large.txt")
cond = np.linalg.cond(A)
print("\nConditional number of matrix A is: {:e}".format(cond))


Conditional number of matrix A is: 5.223236e+25


In [29]:
A_sp = csc_matrix(A) 
x = iterRef(A_sp, B, tol=1e-15)
show_results(x, A_sp, B, "IterRef")
# print(np.linalg.norm(A.dot(x)-b)/np.linalg.norm(b))

res :: A * x - b = -2.207863e-11
iteration   1, norm = 2.922399e-02
iteration   2, norm = 1.145121e-03
iteration   3, norm = 1.091830e-03
iteration   4, norm = 2.710505e-20
The procedure was successful.
IR: A * x - b = -1.442585e-12
number of iteration is : 4
 
for IterRef          Ax-b is 1.005217e-12
